### **API SADVR - Portrait statistique**  
https://www.cen.umontreal.ca/espacedoc/sadvr/  

Le Service d’accès aux données de la Vitrine-Recherche (SADVR) est un service institutionnel de partage de données relatives aux profils des professeur·e·s et à leurs activités académiques ainsi qu'aux expertises et disciplines de recherche des facultés, départements et centres de recherche affiliés à l'Université.

---

#### **Professeurs**

In [2]:
import requests
import json
import pandas as pd
from ast import literal_eval
from collections import Counter
import SADVR_getTables

Import des données

In [3]:
baseURI = 'https://www.recherche.umontreal.ca/vitrine/rest/api/1.7/umontreal'

# Mettre à jour les tables de données 
SADVR_getTables.getAllTables()
SADVR_getTables.updateInfoIndividus()
SADVR_getTables.getAllProfs()

dataIndividus = pd.read_csv('tables/SADVR_infoIndividus.csv')
dataProfs = pd.read_csv('tables/SADVR_professeurs.csv')

Normalisation / filtrage

In [ ]:
dataIndividus = dataIndividus.explode('etablissementsAffilies').reset_index(drop=True)
dataIndividus['etablissementAffilie'] = dataIndividus['etablissementsAffilies'].apply(
    lambda x: x['nom'] if isinstance(x, dict) else None)

dataIndividus = dataIndividus.drop(columns=
        ['etablissementsAffilies', 'fonction', 'courriel', 'visuelOfficiel', 'visuelOfficielGrand']
    ).drop_duplicates()

dataIndividus = dataIndividus[dataIndividus['nom'] != '?_?']

# Séparer les colonnes qui contiennent des données structurées en JSON en muliples colonnes distinctes
def explodeNormalize(df: pd.DataFrame, columns: list, ):
    """
    Cette fonction prend en paramètre un DataFrame et une liste contenant les noms des colonnes à normaliser.
    Elle retourne le DataFrame modifié
    """
    for col in columns:
        dTypeCol = Counter(df[col].apply(lambda x: type(literal_eval(str(x)))).tolist()).most_common(1)[0][0]
        if dTypeCol == list:
            df = df.explode(col).reset_index(drop=True)

        dfTemp = pd.json_normalize(df[col]).add_prefix(f'{col}.') 
        
        df = pd.concat([df, dfTemp], axis=1).drop(col, axis=1)

    return df

# Colonnes à normaliser
columns = ['affiliations', 'expertise', 'expertise.secteursRecherche',
    'expertise.disciplines', 'expertise.motsCles', 'expertise.periodesChronologiques',
    'expertise.phraseCle', 'expertise.continents']

dataProfs = explodeNormalize(dataProfs, columns)

dataProfs = dataProfs.drop(
    columns=["prenom", "nom", "etablissementsAffilies", "visuelOfficiel", "visuelOfficielGrand",
      "courriels", "telephones", "urlVitrine", "nomSAD", "prenomSAD", "in-memoriam",
      "affiliations.courrielInstitutionnel", "affiliations.local", "affiliations.exclusion", "affiliations.exclusionTel", 
      "affiliations.fonction.nom", "affiliations.telephone.numero", "affiliations.telephone.poste"])

data = dataIndividus.merge(dataProfs, how='right')
data

In [ ]:
data.to_csv('./tables/SADVR_professeurs.csv', index=False)